In [ ]:
################################ EDA IMPORTS ###################################
%matplotlib inline
import pandas as pd 
import pandas_profiling # Quick Data Analysis
import numpy as np # Linear Algebra lib
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objs as go # interactive low-level plotting lib https://plot.ly/python/
import plotly.express as px #high-level api wrapper for plotly https://plot.ly/python/plotly-express/#visualize-distributions
# ---------------- Plot libs settings ------------- #
# Pick style of Matplolib plots 
# Different style sheets:-> https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html
# Configure Seaborn Asthetics: -> https://seaborn.pydata.org/tutorial/aesthetics.html?highlight=style
plt.style.use('seaborn-darkgrid')
sns.set(context='notebook', style='darkgrid', palette='colorblind')
# Seting a universal figure size 
plt.rcParams['figure.figsize'] = (10, 6)

# ---------------- Pandas settings ------------- #
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

################################################################################

#################################### ML IMPORTS ################################
### Pre-Processing
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

### Train/Test - Split & CV
from sklearn.model_selection import train_test_split

### Regressors
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

### Pipeline
from sklearn.pipeline import make_pipeline

### Feature Importance
import eli5
from eli5.sklearn import PermutationImportance

### Tuning Hyperparameters
from sklearn.model_selection import GridSearchCV



################################################################################

# Project PUBG 🤖💣💥🔫🤖 

![](https://o.aolcdn.com/images/dims?quality=100&image_uri=http%3A%2F%2Fo.aolcdn.com%2Fhss%2Fstorage%2Fmidas%2Fb0be09f425cc5175fb413bc03c32dd0d%2F206235889%2Fpubg-ed.jpg&client=amp-blogside-v2&signature=88c6b77342cbeb0d25c0dc9d909018136aec1971)

### Load Dataset

In [ ]:
def load_dataset(f, verbose=True, develop=True):
    df = pd.read_csv('../pubg-api/pubg_stats.csv')
    df = reduce_mem_usage(df, verbose=verbose)
    if verbose:
        print('-' * 80)
        print(
        f'''
        -------------------- SHAPE ---------------------
        PUBG Stats: {df.shape}
        ------------------------------------------------
        ''')
    return df

train = load_dataset('../clean-datasets/train.csv')
val = load_dataset('../clean-datasets/validate.csv')
test = load_dataset('../clean-datasets/test.csv')

train.head()

## Feature Engineering

In [ ]:
mask = (train['duration'] < 800) & (train['walkDistance'] == 0)
train[mask]

winPlace == 1 and walkDistance == 0 are cheaters or hacks in glitches

drop outliers:

- high weapons picked up
- high ride distance, no kills
- high walkdistance, no kills
- high heals, no kills


drop team killers



In [ ]:
def feature_engineering(df):
    
    # Remove players that won with no kills, or winPlacePerc > 0.9
    
    
    
    
    df['headKillsRatio'] = df['headshotKills'] / df['kills']
    




## Random Forest

## Light GBM